# Práce s modely rodiny Meta

## Úvod

Tato lekce se zaměří na:

- Prozkoumání dvou hlavních modelů rodiny Meta – Llama 3.1 a Llama 3.2
- Pochopení, k čemu se jednotlivé modely hodí a v jakých situacích je použít
- Ukázka kódu, která předvede jedinečné vlastnosti každého modelu

## Rodina modelů Meta

V této lekci se podíváme na 2 modely z rodiny Meta, někdy označované jako „Llama Herd“ – Llama 3.1 a Llama 3.2

Tyto modely existují v různých variantách a najdete je na tržišti Github Model. Další informace o využití Github Models k [prototypování s AI modely](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Varianty modelů:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*Poznámka: Llama 3 je také dostupná na Github Models, ale v této lekci se jí věnovat nebudeme*


## Llama 3.1

S 405 miliardami parametrů spadá Llama 3.1 do kategorie open source LLM.

Tento model je vylepšením předchozí verze Llama 3 a nabízí:

- Větší kontextové okno – 128k tokenů oproti 8k tokenům
- Vyšší maximální počet výstupních tokenů – 4096 oproti 2048
- Lepší vícejazyčnou podporu – díky navýšení počtu trénovacích tokenů

Díky tomu dokáže Llama 3.1 zvládat složitější případy použití při tvorbě GenAI aplikací, včetně:
- Nativní volání funkcí – možnost volat externí nástroje a funkce mimo workflow LLM
- Lepší výkon při RAG – díky většímu kontextovému oknu
- Generování syntetických dat – možnost vytvářet efektivní data pro úlohy jako je doladění modelu


### Nativní volání funkcí

Llama 3.1 byla doladěna tak, aby byla efektivnější při volání funkcí nebo nástrojů. Má také dva vestavěné nástroje, které dokáže rozpoznat jako vhodné k použití na základě uživatelského zadání. Tyto nástroje jsou:

- **Brave Search** – Lze použít k získání aktuálních informací, například o počasí, pomocí webového vyhledávání
- **Wolfram Alpha** – Lze využít pro složitější matematické výpočty, takže není nutné psát vlastní funkce.

Můžete si také vytvořit vlastní nástroje, které může LLM volat.

V níže uvedeném příkladu kódu:

- Definujeme dostupné nástroje (brave_search, wolfram_alpha) v systémovém promptu.
- Pošleme uživatelský dotaz, který se ptá na počasí v určitém městě.
- LLM odpoví voláním nástroje Brave Search, které bude vypadat takto `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Poznámka: Tento příklad pouze provede volání nástroje. Pokud chcete získat výsledky, je potřeba si zdarma vytvořit účet na stránce Brave API a definovat samotnou funkci.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

I když je Llama 3.1 jazykový model, jednou z jeho omezení je multimodalita. To znamená, že neumí pracovat s různými typy vstupů, například s obrázky jako podněty a poskytovat na ně odpovědi. Právě tato schopnost je jednou z hlavních novinek Llama 3.2. Mezi další vlastnosti patří:

- Multimodalita – dokáže zpracovávat jak textové, tak obrazové podněty
- Varianty od malých po střední (11B a 90B) – umožňují flexibilní nasazení,
- Pouze textové varianty (1B a 3B) – umožňují nasazení modelu na okrajových / mobilních zařízeních a zajišťují nízkou latenci

Podpora multimodality představuje významný krok ve světě open source modelů. Následující ukázka kódu využívá jak obrázek, tak textový podnět, aby získala analýzu obrázku od Llama 3.2 90B.

### Podpora multimodality s Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Učení zde nekončí, pokračujte na své cestě

Po dokončení této lekce se podívejte na naši [sbírku o Generativní AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) a dále rozvíjejte své znalosti v oblasti generativní umělé inteligence!



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho rodném jazyce. Pro kritické informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vzniklé v důsledku použití tohoto překladu.
